In [3]:
#####
# PUT EVENT TIME HERE
#eventOriginTime = obspy.core.utcdatetime.UTCDateTime(2021, 10, 16, 9, 34, 0)
eventOriginTime = obspy.core.utcdatetime.UTCDateTime(2021, 10, 16, 18, 27, 29) # earthquake M 5.3 - 74 km SSW of George Town, Cayman Islands
# 2021-10-16 18:27:29 (UTC)18.701°N 81.727°W10.0 km depth
eventType = "Rocket"
eventType = "Earthquake"
#####

# Quick look at SDS archive
import os
import obspy

# Find the SD card on a Mac
import glob
eventsdirs = glob.glob('/Volumes/*/events')
if len(eventsdirs) == 1:
    TOPDIR = os.path.dirname(eventsdirs[0])
else:
    print('Should be exactly 1 SD card mounted. Found %d. Quitting.' % len(eventsdirs))
    raise KeyboardInterrupt
print("SD card appears to be mounted at ",TOPDIR)

# standard config setup for sensor A trillium, sensor B infraBSUs
locations = ['00', '10']
components = ['HHZ', 'HHN', 'HHE', 'HD1', 'HD2', 'HD3', 'HDF'] # Use HDF for 1-channel infrasound, otherwise HD1,2,3
fix_location = True
YYYY = "%4d" % eventOriginTime.year
JJJ = "%03d" % eventOriginTime.julday
stationdirs = glob.glob(os.path.join(TOPDIR, str(YYYY), '??', '*' ))
if len(stationdirs) == 1:
    station = os.path.basename(stationdirs[0])
    net = os.path.basename(os.path.dirname(stationdirs[0]))
else:
    print('Should be exactly 1 station on the SD card. Found %d. Quitting.' % len(stationdirs))
    raise KeyboardInterrupt
print("Correct network and station appears to be %s.%s" % (net, station))

EVENTDIR = os.path.join(os.getenv('HOME'), 'Dropbox', '%sEvents' % eventType, eventOriginTime.strftime('%Y-%m-%d'))
if not os.path.exists(EVENTDIR):
    os.makedirs(EVENTDIR)
eventfile = os.path.join(EVENTDIR, '%s_event_%s_%s.mseed' % (eventType.lower(), eventOriginTime.strftime('%Y-%m-%d-%H-%M-%S'), station))

# Time windows for rocket event processing. Should work for most regional earthquakes too.
spaceXlanding = False
if spaceXlanding:
    eventDurationSecs = 720
else: 
    eventDurationSecs = 180    
preEventSecs = 120
postEventSecs = 180
taperSecs = 360 # amount of taper to add on each end

taperStartTime = eventOriginTime - preEventSecs - taperSecs
taperEndTime = eventOriginTime + eventDurationSecs + postEventSecs + taperSecs
print('Selecting data from ',taperStartTime,' to ',taperEndTime)

if os.path.exists(eventfile):
    print("Loading %s" % eventfile)
    st = obspy.core.read(eventfile)
else:
    st = obspy.core.Stream()
    for location in locations:
        for component in components: 
            if fixwrong:
                MSEEDDIR = os.path.join(TOPDIR, YYYY, wrongnet, wrongstation, '%s.D' % component)
                mseedfile = os.path.join(MSEEDDIR, '.'.join([wrongnet, wrongstation, location, component, 'D', YYYY, JJJ]))
            else:
                MSEEDDIR = os.path.join(TOPDIR, YYYY, net, station, '%s.D' % component)
                mseedfile = os.path.join(MSEEDDIR, '.'.join([net, station, location, component, 'D', YYYY, JJJ]))
            #print('Trying to load %s' % mseedfile)
            if not os.path.exists(mseedfile):
                mseedfile = mseedfile + '.miniseed'
            if os.path.exists(mseedfile):
                print("Loading %s" % mseedfile)
                st0 = obspy.core.read(mseedfile)
                st0.trim(starttime=taperStartTime, endtime=taperEndTime)
                for tr in st0:
                    st.append(tr)
    if len(st)>0:
        st.write(eventfile)
if len(st)>0:    
    print('Data loaded.')
    st.plot(outfile = eventfile.replace('.mseed', '_raw.png'), equal_scale=False)
else:
    print("No waveform data found. Quitting.")
    raise KeyboardInterrupt

# Processing
print('Processing.')
st.detrend()
durationSecs = st[0].stats.delta * (st[0].stats.npts-1)
taperFraction = taperSecs / durationSecs
st.taper(taperFraction, type='hann', max_length=None, side='both')
st.filter("bandpass", freqmin=0.5, freqmax=20.0) 
st.trim(starttime=taperStartTime+taperSecs, endtime=taperEndTime-taperSecs)

# plot seismic channels
print('Plotting and saving to PNG files')
seismic = st.select(channel='?H?')
seismic.plot(outfile = eventfile.replace('.mseed', '_seismic.png'), equal_scale=False)

# plot infrasound channels
infrasound = st.select(channel='?D?')
infrasound.plot(outfile = eventfile.replace('.mseed', '_infrasound.png'), equal_scale=False)

# SOH data
soh_st = obspy.core.Stream()
SOHDIR = os.path.join(TOPDIR, 'DAYS', station, 'soh')
sohfile = os.path.join(SOHDIR, '.'.join([station, net, 'D0.SOH.S', YYYY, JJJ]))
soh_eventfile = eventfile.replace('.mseed', '_soh.mseed')
if os.path.exists(soh_eventfile):
    soh_st = obspy.core.read(soh_eventfile)
else:
    if os.path.exists(sohfile):
        soh_st = obspy.core.read(sohfile)
        soh_st.write(soh_eventfile)
if soh_st:
    soh_st.plot(outfile = eventfile.replace('.mseed', '_soh.png'), equal_scale=False)

# Station XML
RESPONSEDIR = os.path.join(TOPDIR, 'responses')
xmlfiles = sorted(glob.glob(os.path.join(RESPONSEDIR, '%s.%s*.xml' % (net, station))))
if len(xmlfiles)>0:
    xmlfile = xmlfiles[-1]
    print('Best Station XML file is %s' % xmlfile)               
    os.system('cp %s %s' % (xmlfile, EVENTDIR))
                  
print('Done')

SD card appears to be mounted at  /Volumes/CENTAUR
Correct network and station appears to be FL.RB383
Selecting data from  2021-10-16T18:19:29.000000Z  to  2021-10-16T18:39:29.000000Z
No waveform data found. Quitting.


KeyboardInterrupt: 